In [1]:
import torch
import torch.nn as nn

In [61]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Get the padding size to make it same padding
class PaddedConv2d(nn.Module):

    def __init__(self, conv2d, pad2d_type):
            """
            Parameters
            ---

            * conv2d : torch.nn.Conv2d
                a convolutional layer used in this PaddedConv2d

            * pad2d_type : type
                a padding layer from torch.nn. I don't want the
                instance itself since it needs the padding size to be created;
                instead, I want the callable which returns an instance of 
                a padding layer, which will be created on the fly during the 
                "forward" pass.
            """

            super().__init__(self)
            self.conv2d = conv2d
            self.pad2d_type = pad2d_type

    def forward(self, in):
        """
        Parameters
        ---
        * in : torch.Tensor
             the input tensor to be padded and then convolved. Shape (batch_size, channels, rows, cols)
        """
        # computing padding size:
        pad_h = torch.ceil((self.conv2d.kernel[0] - in.shape[2] * (1 - self.conv2d.stride[0]) - self.conv2d.stride[0]) / 2)
        pad_w = torch.ceil((self.conv2d.kernel[1] - in.shape[3] * (1 - self.conv2d.stride[1]) - self.conv2d.stride[1]) / 2)
        padder = self.pad2d_type((pad_w, pad_w, pad_h, pad_h))
        return self.conv2d(padder(in))

SyntaxError: invalid syntax (<ipython-input-5-d030bc1b9a38>, line 24)

In [13]:
# Calculate manually the padding layer
#ks = 7 , stride =1
# formula for same padding with stride 1 is p = (k-1)/2 , here = (7-1)/2 = 3
n = 512
pad = 3
stride = 1
ks = 7
out  = (n + 2*pad - ks) // stride + 1
print(out)

512


In [22]:
# Calculate manually the padding layer
#ks = 3 , stride =1
# formula for same padding with stride 1 is p = (k-1)/2 , here = (3-1)/2 = 1
n = 256
pad = 1
stride = 1
ks = 3
out  = (n + 2*pad - ks) // stride + 1
print(out)

256


In [28]:
# Calculate manually the padding layer
#ks = 3 , stride =2
n = 512
pad = 1
stride = 2
ks = 3
out  = (n + 2*pad - ks) // stride + 1
print(out)

256


In [15]:
# Batch Norm Examples::
# With Learnable Parameters
# Number of channel - here it is 100/ it is also referred as number of filter
m = nn.BatchNorm2d(100)
# Without Learnable Parameters
#m = nn.BatchNorm2d(100, affine=False)
input = torch.randn(20, 100, 35, 45)
output = m(input)

In [17]:
output.shape

torch.Size([20, 100, 35, 45])

In [53]:
# Image alteration
import numpy as np 
rotsd=[0.0, 5.0]
scalesd=[0.0, 0.1]
transsd=[0.1, 0.1]
warpsd=[0.001, 0.005, 0.001, 0.01]
Hc = 64
Wc =64
def sample_tps_w(Hc, Wc, warpsd, rotsd, scalesd, transsd):
  """
  Returns randomly sampled TPS-grid params of size (Hc*Wc+3)x2.

  Params:
    WARPSD: 2-tuple
    {ROT/SCALE/TRANS}-SD: 1-tuple of standard devs.
  """
  Nc = Hc * Wc  # no of control-pots
  # non-linear component:
  mask = (np.random.rand(Nc, 2) > 0.5).astype(np.float32)
  W = warpsd[0] * np.random.randn(Nc, 2) + \
      warpsd[1] * (mask * np.random.randn(Nc, 2))
  # affine component:
  rnd = np.random.randn
  rot = np.deg2rad(rnd() * rotsd)
  sc = 1.0 + rnd() * scalesd
  aff = [[transsd*rnd(),      transsd*rnd()],
         [sc * np.cos(rot),   sc * -np.sin(rot)],
         [sc * np.sin(rot),   sc * np.cos(rot)]]
  W = np.r_[W, aff]
  return W

In [55]:
W = sample_tps_w(Hc, Wc, warpsd, rotsd, scalesd, transsd)

In [58]:
64*64

4096

In [57]:
W.shape

(4099, 2)

In [46]:
rnd = np.random.randint(100)

In [47]:
rnd

32

In [34]:
rnd()*rotsd

TypeError: can't multiply sequence by non-int of type 'float'

In [54]:
vertical_points=10
horizontal_points=10
rotsd=0.0
scalesd=0.0
transsd=0.1
warpsd=(0.001, 0.005)
